# Charlottesville Open Data Porttal: Crime Data

## Import Relevant Librarires & Set WD

In [1]:
import requests
import pandas as pd
import numpy as np
import pickle
import googlemaps
import plotly.express as px
from geopy.geocoders import Nominatim
import time
from dotenv import load_dotenv
import os

env_path = os.path.join("..", ".env")
#print("Looking for .env file at:", os.path.abspath(env_path))
load_dotenv(dotenv_path=env_path)

# Retrieve the variables
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
WORKING_DIR = os.getenv("WORKING_DIR")

if GOOGLE_API_KEY is None or WORKING_DIR is None:
    raise RuntimeError("No configuration found. Please ensure that your .env file exists with the required variables, or create a local config.py file.")

#print("Google API Key:", GOOGLE_API_KEY)
#print("Working Directory:", WORKING_DIR)

## Import Data

URL: https://opendata.charlottesville.org/datasets/charlottesville::crime-data/about

In [2]:
# Define the endpoint and initial parameters.
url = "https://gisweb.charlottesville.org/arcgis/rest/services/OpenData_2/MapServer/6/query"
params = {
    "where": "1=1",
    "outFields": "*",
    "outSR": 4326,
    "f": "json",
    "resultOffset": 0,       # Starting index for results
    "resultRecordCount": 10000  # Maximum records per batch (if allowed by the API)
}

records = []

while True:
    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json()
    
    # Get the current batch of features.
    features = data.get("features", [])
    if not features:
        break

    # Extract the attributes from the features.
    batch_records = [feature.get("attributes", {}) for feature in features]
    records.extend(batch_records)
    
    # If the batch size is less than the requested count, we reached the end.
    if len(features) < params["resultRecordCount"]:
        break

    # Update the resultOffset for the next batch.
    params["resultOffset"] += params["resultRecordCount"]

# Create a DataFrame from the combined records.
df = pd.DataFrame(records)
print("Total records imported:", len(df))


Total records imported: 26159


In [3]:
df.head()

,RecordID,Offense,IncidentID,BlockNumber,StreetName,Agency,DateReported,HourReported,ReportingOfficer
0,1,Assault Aggravated,202600000005,1600,MERIDIAN ST,CPD,1767244082000,0008,"Ritter, Kobe"
1,2,Assault Simple,202500048451,100,HURST LN,CPD,1767229712000,2008,"Vlasis, Christopher"
2,3,Domestic Disturbance,202500048448,600,E WATER ST,CPD,1767227911000,1938,"Ritter, Kobe"
3,4,Hit and Run,202500048437,900,MARTIN ST,CPD,1767222092000,1801,"McCall, Matthew"
4,5,Larceny - Shoplifitng,202500048432,200,E MAIN ST,CPD,1767219267000,1714,"Ritter, Kobe"


In [4]:
df.columns

Index(['RecordID', 'Offense', 'IncidentID', 'BlockNumber', 'StreetName',
       'Agency', 'DateReported', 'HourReported', 'ReportingOfficer'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26159 entries, 0 to 26158
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   RecordID          26159 non-null  int64 
 1   Offense           26159 non-null  object
 2   IncidentID        26159 non-null  object
 3   BlockNumber       26159 non-null  object
 4   StreetName        26159 non-null  object
 5   Agency            26159 non-null  object
 6   DateReported      26159 non-null  int64 
 7   HourReported      26159 non-null  object
 8   ReportingOfficer  26147 non-null  object
dtypes: int64(2), object(7)
memory usage: 1.8+ MB


### Missing Values

In [6]:
def missing_percentage(df):
    """
    Returns a DataFrame with the count and percentage of missing values for each column.
    
    Parameters:
        df (pd.DataFrame): The input DataFrame.
    
    Returns:
        pd.DataFrame: A DataFrame with columns 'MissingCount' and 'MissingPercentage'.
    """
    total_rows = len(df)
    missing_count = df.isnull().sum()
    missing_percent = (missing_count / total_rows) * 100
    
    result = pd.DataFrame({
        'MissingCount': missing_count,
        'MissingPercentage': missing_percent.round(2)
    })
    
    return result

# Example usage:
missing_info = missing_percentage(df)
print(missing_info)


                  MissingCount  MissingPercentage
RecordID                     0               0.00
Offense                      0               0.00
IncidentID                   0               0.00
BlockNumber                  0               0.00
StreetName                   0               0.00
Agency                       0               0.00
DateReported                 0               0.00
HourReported                 0               0.00
ReportingOfficer            12               0.05


## Data Preparation

### Data Conversion

In [7]:
def convert_and_categorize_datereported(df, col='DateReported'):
    """
    Converts an epoch (in milliseconds) date column into a timezone-aware datetime,
    converts it to Eastern Time, and then creates columns for day-of-week, weekend flag,
    season, 12-hour formatted time, and time-of-day. Keeps both a real datetime column
    (Date) and a string representation (DateString).
    """
    import pandas as pd

    # Convert the epoch timestamp to a pandas datetime object as UTC, then convert to US/Eastern.
    df["DateReported_dt"] = (
        pd.to_datetime(df[col], unit="ms", errors="coerce", utc=True)
        .dt.tz_convert("US/Eastern")
    )
    
    # Create a true datetime column (remove timezone if you prefer naive datetime)
    df["Date"] = df["DateReported_dt"].dt.tz_localize(None)

    # Optionally, create a separate string-formatted column
    df["DateString"] = df["DateReported_dt"].dt.strftime("%Y-%m-%d %I:%M:%S %p")
    
    # DayOfWeek
    df["DayOfWeek"] = df["DateReported_dt"].dt.day_name()
    
    # Weekend
    df["Weekend"] = df["DayOfWeek"].isin(["Saturday", "Sunday"])
    
    # Helper function to determine the season
    def get_season(dt):
        month = dt.month
        day = dt.day
        # Approximate Northern Hemisphere season boundaries:
        if (month == 12 and day >= 21) or (month < 3) or (month == 3 and day < 20):
            return "Winter"
        elif (month == 3 and day >= 20) or (month < 6) or (month == 6 and day < 21):
            return "Spring"
        elif (month == 6 and day >= 21) or (month < 9) or (month == 9 and day < 22):
            return "Summer"
        else:
            return "Autumn"
    
    df["Season"] = df["DateReported_dt"].apply(get_season)
    
    # 12-hour formatted time (HourAMPM)
    df["HourAMPM"] = df["DateReported_dt"].dt.strftime("%I:%M %p")
    
    # Extract hour (0-23) for time-of-day
    df["HourValue"] = df["DateReported_dt"].dt.hour
    
    def time_of_day(hour):
        if pd.isnull(hour):
            return None
        if 5 <= hour < 12:
            return "Morning"
        elif 12 <= hour < 17:
            return "Afternoon"
        elif 17 <= hour < 21:
            return "Evening"
        else:
            return "Night"
    
    df["TimeOfDay"] = df["HourValue"].apply(time_of_day)
    
    # Drop columns you no longer need
    df.drop(columns=[col, "DateReported_dt", "HourValue"], inplace=True)
    
    return df

# Example usage:
df = convert_and_categorize_datereported(df)
df.head()


,RecordID,Offense,IncidentID,BlockNumber,StreetName,Agency,HourReported,ReportingOfficer,Date,DateString,DayOfWeek,Weekend,Season,HourAMPM,TimeOfDay
0,1,Assault Aggravated,202600000005,1600,MERIDIAN ST,CPD,0008,"Ritter, Kobe",2026-01-01 00:08:02,2026-01-01 12:08:02 AM,Thursday,False,Winter,12:08 AM,Night
1,2,Assault Simple,202500048451,100,HURST LN,CPD,2008,"Vlasis, Christopher",2025-12-31 20:08:32,2025-12-31 08:08:32 PM,Wednesday,False,Winter,08:08 PM,Evening
2,3,Domestic Disturbance,202500048448,600,E WATER ST,CPD,1938,"Ritter, Kobe",2025-12-31 19:38:31,2025-12-31 07:38:31 PM,Wednesday,False,Winter,07:38 PM,Evening
3,4,Hit and Run,202500048437,900,MARTIN ST,CPD,1801,"McCall, Matthew",2025-12-31 18:01:32,2025-12-31 06:01:32 PM,Wednesday,False,Winter,06:01 PM,Evening
4,5,Larceny - Shoplifitng,202500048432,200,E MAIN ST,CPD,1714,"Ritter, Kobe",2025-12-31 17:14:27,2025-12-31 05:14:27 PM,Wednesday,False,Winter,05:14 PM,Evening


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26159 entries, 0 to 26158
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   RecordID          26159 non-null  int64         
 1   Offense           26159 non-null  object        
 2   IncidentID        26159 non-null  object        
 3   BlockNumber       26159 non-null  object        
 4   StreetName        26159 non-null  object        
 5   Agency            26159 non-null  object        
 6   HourReported      26159 non-null  object        
 7   ReportingOfficer  26147 non-null  object        
 8   Date              26159 non-null  datetime64[ns]
 9   DateString        26159 non-null  object        
 10  DayOfWeek         26159 non-null  object        
 11  Weekend           26159 non-null  bool          
 12  Season            26159 non-null  object        
 13  HourAMPM          26159 non-null  object        
 14  TimeOfDay         2615

### Coordinates

In [9]:
# Combine BlockNumber and StreetName into a full address string
df['FullStreet'] = df['BlockNumber'].astype(str) + " " + df['StreetName']

# --- Step 1: Setup the Google Maps Client and Cache ---
gmaps = googlemaps.Client(key=GOOGLE_API_KEY)

# Try to load cached geocoding results if available.
try:
    with open("geocode_cache_google.pkl", "rb") as f:
        street_coords = pickle.load(f)
    print("Loaded geocode cache.")
except FileNotFoundError:
    street_coords = {}
    print("No cache found; starting fresh.")

def geocode_address(address):
    try:
        # Append city and state to help geocoding
        result = gmaps.geocode(f"{address}, Charlottesville, VA")
        if result:
            lat = result[0]['geometry']['location']['lat']
            lon = result[0]['geometry']['location']['lng']
            return lat, lon
    except Exception as e:
        print(f"Error geocoding {address}: {e}")
    return None, None

# --- Step 2: Geocode Only New Addresses ---
unique_addresses = df['FullStreet'].unique()
for address in unique_addresses:
    if address not in street_coords:
        lat, lon = geocode_address(address)
        street_coords[address] = (lat, lon)
        print(f"Geocoded {address}: {lat}, {lon}")

# Save the updated cache to disk.
with open("geocode_cache_google.pkl", "wb") as f:
    pickle.dump(street_coords, f)
    print("Geocode cache updated and saved.")

# --- Step 3: Map Coordinates Back to DataFrame ---
df['lat'] = df['FullStreet'].map(lambda s: street_coords.get(s, (None, None))[0])
df['lon'] = df['FullStreet'].map(lambda s: street_coords.get(s, (None, None))[1])

df.head()

Loaded geocode cache.
Geocode cache updated and saved.


,RecordID,Offense,IncidentID,BlockNumber,StreetName,Agency,HourReported,ReportingOfficer,Date,DateString,DayOfWeek,Weekend,Season,HourAMPM,TimeOfDay,FullStreet,lat,lon
0,1,Assault Aggravated,202600000005,1600,MERIDIAN ST,CPD,0008,"Ritter, Kobe",2026-01-01 00:08:02,2026-01-01 12:08:02 AM,Thursday,False,Winter,12:08 AM,Night,1600 MERIDIAN ST,38.015215,-78.481585
1,2,Assault Simple,202500048451,100,HURST LN,CPD,2008,"Vlasis, Christopher",2025-12-31 20:08:32,2025-12-31 08:08:32 PM,Wednesday,False,Winter,08:08 PM,Evening,100 HURST LN,38.018332,-78.498265
2,3,Domestic Disturbance,202500048448,600,E WATER ST,CPD,1938,"Ritter, Kobe",2025-12-31 19:38:31,2025-12-31 07:38:31 PM,Wednesday,False,Winter,07:38 PM,Evening,600 E WATER ST,38.028757,-78.477608
3,4,Hit and Run,202500048437,900,MARTIN ST,CPD,1801,"McCall, Matthew",2025-12-31 18:01:32,2025-12-31 06:01:32 PM,Wednesday,False,Winter,06:01 PM,Evening,900 MARTIN ST,38.037759,-78.464863
4,5,Larceny - Shoplifitng,202500048432,200,E MAIN ST,CPD,1714,"Ritter, Kobe",2025-12-31 17:14:27,2025-12-31 05:14:27 PM,Wednesday,False,Winter,05:14 PM,Evening,200 E MAIN ST,38.030572,-78.480764


### Neighborhood

In [10]:
# Initialize the Google Maps client.
gmaps = googlemaps.Client(key=GOOGLE_API_KEY)

# Try to load a persistent neighborhood cache if available.
try:
    with open("neighborhood_cache.pkl", "rb") as f:
        neighborhood_cache = pickle.load(f)
    print("Loaded neighborhood cache.")
except FileNotFoundError:
    neighborhood_cache = {}
    print("No neighborhood cache found; starting fresh.")

def get_neighborhood(lat, lon):
    """
    Reverse geocodes a latitude and longitude to retrieve the neighborhood.
    Caches results to speed up subsequent lookups.
    """
    key = (lat, lon)
    if key in neighborhood_cache:
        return neighborhood_cache[key]
    
    try:
        result = gmaps.reverse_geocode((lat, lon))
        if result:
            # Look for the 'neighborhood' component in the first result.
            for component in result[0]['address_components']:
                if 'neighborhood' in component['types']:
                    neighborhood = component['long_name']
                    neighborhood_cache[key] = neighborhood
                    return neighborhood
            # If no neighborhood is found, return "N/A"
            neighborhood_cache[key] = "N/A"
            return "N/A"
    except Exception as e:
        print(f"Error getting neighborhood for {lat}, {lon}: {e}")
        return "N/A"
    
    # Pause to respect usage limits.
    time.sleep(1)

# Example usage: Apply to your DataFrame 'df' that already has 'lat' and 'lon' columns.
df['neighborhood'] = df.apply(lambda row: get_neighborhood(row['lat'], row['lon']), axis=1)

# Save the updated neighborhood cache.
with open("neighborhood_cache.pkl", "wb") as f:
    pickle.dump(neighborhood_cache, f)

df.head()


Loaded neighborhood cache.


Error getting neighborhood for nan, nan: HTTP Error: 400


Error getting neighborhood for nan, nan: HTTP Error: 400
Error getting neighborhood for nan, nan: HTTP Error: 400


Error getting neighborhood for nan, nan: HTTP Error: 400
Error getting neighborhood for nan, nan: HTTP Error: 400
Error getting neighborhood for nan, nan: HTTP Error: 400
Error getting neighborhood for nan, nan: HTTP Error: 400
Error getting neighborhood for nan, nan: HTTP Error: 400


Error getting neighborhood for nan, nan: HTTP Error: 400


...

Error getting neighborhood for nan, nan: HTTP Error: 400
Error getting neighborhood for nan, nan: HTTP Error: 400
Error getting neighborhood for nan, nan: HTTP Error: 400
Error getting neighborhood for nan, nan: HTTP Error: 400
Error getting neighborhood for nan, nan: HTTP Error: 400


Error getting neighborhood for nan, nan: HTTP Error: 400
Error getting neighborhood for nan, nan: HTTP Error: 400
Error getting neighborhood for nan, nan: HTTP Error: 400


Error getting neighborhood for nan, nan: HTTP Error: 400
Error getting neighborhood for nan, nan: HTTP Error: 400
Error getting neighborhood for nan, nan: HTTP Error: 400
Error getting neighborhood for nan, nan: HTTP Error: 400
Error getting neighborhood for nan, nan: HTTP Error: 400


Error getting neighborhood for nan, nan: HTTP Error: 400
Error getting neighborhood for nan, nan: HTTP Error: 400


,RecordID,Offense,IncidentID,BlockNumber,StreetName,Agency,HourReported,ReportingOfficer,Date,DateString,DayOfWeek,Weekend,Season,HourAMPM,TimeOfDay,FullStreet,lat,lon,neighborhood
0,1,Assault Aggravated,202600000005,1600,MERIDIAN ST,CPD,0008,"Ritter, Kobe",2026-01-01 00:08:02,2026-01-01 12:08:02 AM,Thursday,False,Winter,12:08 AM,Night,1600 MERIDIAN ST,38.015215,-78.481585,Belmont
1,2,Assault Simple,202500048451,100,HURST LN,CPD,2008,"Vlasis, Christopher",2025-12-31 20:08:32,2025-12-31 08:08:32 PM,Wednesday,False,Winter,08:08 PM,Evening,100 HURST LN,38.018332,-78.498265,Johnson Village
2,3,Domestic Disturbance,202500048448,600,E WATER ST,CPD,1938,"Ritter, Kobe",2025-12-31 19:38:31,2025-12-31 07:38:31 PM,Wednesday,False,Winter,07:38 PM,Evening,600 E WATER ST,38.028757,-78.477608,North Downtown
3,4,Hit and Run,202500048437,900,MARTIN ST,CPD,1801,"McCall, Matthew",2025-12-31 18:01:32,2025-12-31 06:01:32 PM,Wednesday,False,Winter,06:01 PM,Evening,900 MARTIN ST,38.037759,-78.464863,Locust Grove
4,5,Larceny - Shoplifitng,202500048432,200,E MAIN ST,CPD,1714,"Ritter, Kobe",2025-12-31 17:14:27,2025-12-31 05:14:27 PM,Wednesday,False,Winter,05:14 PM,Evening,200 E MAIN ST,38.030572,-78.480764,Downtown Mall


### Zip

In [11]:
# Initialize the geocoder.
geolocator = Nominatim(user_agent="zip_lookup")

# Try to load a persistent ZIP cache if available.
try:
    with open("zip_cache.pkl", "rb") as f:
        zip_cache = pickle.load(f)
    print("Loaded zip cache.")
except FileNotFoundError:
    zip_cache = {}
    print("No zip cache found; starting fresh.")

def get_zip(lat, lon):
    """
    Reverse geocodes a latitude and longitude to retrieve the ZIP code.
    Caches results to speed up subsequent lookups.
    """
    key = (lat, lon)
    if key in zip_cache:
        return zip_cache[key]
    
    try:
        location = geolocator.reverse((lat, lon), exactly_one=True, timeout=10)
        address = location.raw.get('address', {})
        postal_code = address.get('postcode', "N/A")
    except Exception as e:
        print(f"Error getting ZIP for {lat}, {lon}: {e}")
        postal_code = "N/A"
    
    zip_cache[key] = postal_code
    # Pause to respect the usage policy.
    time.sleep(1)
    return postal_code

# Apply the function to each row in the DataFrame.
df['zip'] = df.apply(lambda row: get_zip(row['lat'], row['lon']), axis=1)

# Save the updated ZIP cache.
with open("zip_cache.pkl", "wb") as f:
    pickle.dump(zip_cache, f)

df.head()


Loaded zip cache.
Error getting ZIP for nan, nan: Must be a coordinate pair or Point


Error getting ZIP for nan, nan: Must be a coordinate pair or Point


Error getting ZIP for nan, nan: Must be a coordinate pair or Point


Error getting ZIP for nan, nan: Must be a coordinate pair or Point


Error getting ZIP for nan, nan: Must be a coordinate pair or Point


...

Error getting ZIP for nan, nan: Must be a coordinate pair or Point


Error getting ZIP for nan, nan: Must be a coordinate pair or Point


Error getting ZIP for nan, nan: Must be a coordinate pair or Point


Error getting ZIP for nan, nan: Must be a coordinate pair or Point


,RecordID,Offense,IncidentID,BlockNumber,StreetName,Agency,HourReported,ReportingOfficer,Date,DateString,DayOfWeek,Weekend,Season,HourAMPM,TimeOfDay,FullStreet,lat,lon,neighborhood,zip
0,1,Assault Aggravated,202600000005,1600,MERIDIAN ST,CPD,0008,"Ritter, Kobe",2026-01-01 00:08:02,2026-01-01 12:08:02 AM,Thursday,False,Winter,12:08 AM,Night,1600 MERIDIAN ST,38.015215,-78.481585,Belmont,22903
1,2,Assault Simple,202500048451,100,HURST LN,CPD,2008,"Vlasis, Christopher",2025-12-31 20:08:32,2025-12-31 08:08:32 PM,Wednesday,False,Winter,08:08 PM,Evening,100 HURST LN,38.018332,-78.498265,Johnson Village,22903
2,3,Domestic Disturbance,202500048448,600,E WATER ST,CPD,1938,"Ritter, Kobe",2025-12-31 19:38:31,2025-12-31 07:38:31 PM,Wednesday,False,Winter,07:38 PM,Evening,600 E WATER ST,38.028757,-78.477608,North Downtown,22902
3,4,Hit and Run,202500048437,900,MARTIN ST,CPD,1801,"McCall, Matthew",2025-12-31 18:01:32,2025-12-31 06:01:32 PM,Wednesday,False,Winter,06:01 PM,Evening,900 MARTIN ST,38.037759,-78.464863,Locust Grove,22901
4,5,Larceny - Shoplifitng,202500048432,200,E MAIN ST,CPD,1714,"Ritter, Kobe",2025-12-31 17:14:27,2025-12-31 05:14:27 PM,Wednesday,False,Winter,05:14 PM,Evening,200 E MAIN ST,38.030572,-78.480764,Downtown Mall,22902


In [12]:
missing_info = missing_percentage(df)
print(missing_info)


                  MissingCount  MissingPercentage
RecordID                     0               0.00
Offense                      0               0.00
IncidentID                   0               0.00
BlockNumber                  0               0.00
StreetName                   0               0.00
Agency                       0               0.00
HourReported                 0               0.00
ReportingOfficer            12               0.05
Date                         0               0.00
DateString                   0               0.00
DayOfWeek                    0               0.00
Weekend                      0               0.00
Season                       0               0.00
HourAMPM                     0               0.00
TimeOfDay                    0               0.00
FullStreet                   0               0.00
lat                        378               1.45
lon                        378               1.45
neighborhood                 0               0.00


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26159 entries, 0 to 26158
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   RecordID          26159 non-null  int64         
 1   Offense           26159 non-null  object        
 2   IncidentID        26159 non-null  object        
 3   BlockNumber       26159 non-null  object        
 4   StreetName        26159 non-null  object        
 5   Agency            26159 non-null  object        
 6   HourReported      26159 non-null  object        
 7   ReportingOfficer  26147 non-null  object        
 8   Date              26159 non-null  datetime64[ns]
 9   DateString        26159 non-null  object        
 10  DayOfWeek         26159 non-null  object        
 11  Weekend           26159 non-null  bool          
 12  Season            26159 non-null  object        
 13  HourAMPM          26159 non-null  object        
 14  TimeOfDay         2615

In [14]:
# Now drop the original BlockNumber and StreetName columns
df.drop(columns=['BlockNumber', 'StreetName','HourAMPM','HourReported','DateString'], inplace=True)
df["zip"] = df["zip"].astype(str)

# Fill all missing values in the DataFrame with "N/A"
df.replace({None: "N/A", np.nan: "N/A", "": "N/A"}, inplace=True)
df.head()

,RecordID,Offense,IncidentID,Agency,ReportingOfficer,Date,DayOfWeek,Weekend,Season,TimeOfDay,FullStreet,lat,lon,neighborhood,zip
0,1,Assault Aggravated,202600000005,CPD,"Ritter, Kobe",2026-01-01 00:08:02,Thursday,False,Winter,Night,1600 MERIDIAN ST,38.015215,-78.481585,Belmont,22903
1,2,Assault Simple,202500048451,CPD,"Vlasis, Christopher",2025-12-31 20:08:32,Wednesday,False,Winter,Evening,100 HURST LN,38.018332,-78.498265,Johnson Village,22903
2,3,Domestic Disturbance,202500048448,CPD,"Ritter, Kobe",2025-12-31 19:38:31,Wednesday,False,Winter,Evening,600 E WATER ST,38.028757,-78.477608,North Downtown,22902
3,4,Hit and Run,202500048437,CPD,"McCall, Matthew",2025-12-31 18:01:32,Wednesday,False,Winter,Evening,900 MARTIN ST,38.037759,-78.464863,Locust Grove,22901
4,5,Larceny - Shoplifitng,202500048432,CPD,"Ritter, Kobe",2025-12-31 17:14:27,Wednesday,False,Winter,Evening,200 E MAIN ST,38.030572,-78.480764,Downtown Mall,22902


In [15]:
missing_info = missing_percentage(df)
print(missing_info)

                  MissingCount  MissingPercentage
RecordID                     0                0.0
Offense                      0                0.0
IncidentID                   0                0.0
Agency                       0                0.0
ReportingOfficer             0                0.0
Date                         0                0.0
DayOfWeek                    0                0.0
Weekend                      0                0.0
Season                       0                0.0
TimeOfDay                    0                0.0
FullStreet                   0                0.0
lat                          0                0.0
lon                          0                0.0
neighborhood                 0                0.0
zip                          0                0.0


In [16]:
df.columns

Index(['RecordID', 'Offense', 'IncidentID', 'Agency', 'ReportingOfficer',
       'Date', 'DayOfWeek', 'Weekend', 'Season', 'TimeOfDay', 'FullStreet',
       'lat', 'lon', 'neighborhood', 'zip'],
      dtype='object')

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26159 entries, 0 to 26158
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   RecordID          26159 non-null  int64         
 1   Offense           26159 non-null  object        
 2   IncidentID        26159 non-null  object        
 3   Agency            26159 non-null  object        
 4   ReportingOfficer  26159 non-null  object        
 5   Date              26159 non-null  datetime64[ns]
 6   DayOfWeek         26159 non-null  object        
 7   Weekend           26159 non-null  bool          
 8   Season            26159 non-null  object        
 9   TimeOfDay         26159 non-null  object        
 10  FullStreet        26159 non-null  object        
 11  lat               26159 non-null  object        
 12  lon               26159 non-null  object        
 13  neighborhood      26159 non-null  object        
 14  zip               2615

### Export Clean Data

In [18]:
# Determine the project root (assuming papermill is run from the repository root)
project_root = os.getcwd()
print("Project Root:", project_root)

# Set the data directory within the project root
data_dir = os.path.join(project_root, "data")

# Ensure the "data" directory exists (create it if it doesn't)
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# Export the DataFrame to CSV in the "data" folder located in the project root
csv_path = os.path.join(data_dir, "charlottesville_crime_incidents.csv")
df.to_csv(csv_path, index=False)
print("Data saved to:", csv_path)

# Construct the path to your Excel file
excel_path = os.path.join(data_dir, "charlottesville_crime_incidents.xlsx")

# Export the DataFrame to Excel
df.to_excel(excel_path, index=False)
print("Excel saved to:", excel_path)


Project Root: /home/runner/work/crime-data/crime-data


Data saved to: /home/runner/work/crime-data/crime-data/data/charlottesville_crime_incidents.csv


Excel saved to: /home/runner/work/crime-data/crime-data/data/charlottesville_crime_incidents.xlsx
